In [2]:
#โฟลเดอร์
import os
import pandas as pd
from tqdm import tqdm

def read_files_in_fraction(directory_path, fraction_start, fraction_end):
    # Get a list of file names in the directory
    file_names = os.listdir(directory_path)

    # Calculate the number of files to read based on the fraction
    num_files_to_read_start = int(len(file_names) * fraction_start)
    num_files_to_read_end = int(len(file_names) * fraction_end)

    # Initialize an empty list to store individual DataFrames
    dataframes = []

    # Use tqdm to create a progress bar for reading files
    for file_name in tqdm(file_names[num_files_to_read_start:num_files_to_read_end], desc="Reading files", unit="file"):
        if file_name.endswith('.csv'):  # Adjust the file extension based on your files (e.g., .csv, .xlsx, etc.)
            file_path = os.path.join(directory_path, file_name)
            df2 = pd.read_csv(file_path)  # Replace 'read_csv' with 'read_excel' for Excel files
            dataframes.append(df2)

    # Concatenate all the individual DataFrames into a single DataFrame
    df3 = pd.concat(dataframes, ignore_index=True)

    return df3

# Replace 'directory_path' with the path to the directory containing your files
directory_path = 'E:\\CUSC Datalake\\cham4'  #<<<<<<<<<<<<<

# Read 1/4 of the files
df = read_files_in_fraction(directory_path, 0, 0.25)  #<<<<<<<<<<<<<

file_save_name = r"E:\Sort Data\cham4_intensity\cham4_intensity_test.csv"  #<<<<<<<<<<<<<

# Filter rows where "id" column contains "multi_sensor"
df = df[df['id'].str.contains('intensity')]

# Convert 'value' column to numeric (float) type if it's not already done
df['value'] = pd.to_numeric(df['value'], errors='coerce')

#extract id_floor
df['id_floor'] = df['id'].str.extract(r'(fl(?:m|\d{1,2}[a-zA-Z]?))')

#sort
df = df.sort_values(by=['id_floor','time']) 

# Remove the 'id' column
#df = df.drop('id', axis=1)

# Convert 'time' column to pandas datetime object
df['time'] = pd.to_datetime(df['time'])

# Round the minute to the nearest minute
df['time'] = df['time'].dt.round('1min')

# Group by 'time' and calculate the average temperature for each unique timestamp
# df['value'] = df.groupby(['id_floor', 'time'])['value'].transform('mean')

# Format the 'value' column to two decimal places
# df['value'] = df['value'].round(2)

# Drop rows with the same time
# df = df.drop_duplicates(subset=['time', 'id_floor'], keep='first')

# rename
df.rename(columns={'value': 'intensity'}, inplace=True)

df.to_csv(file_save_name, index=False)



Reading files:   0%|          | 0/732 [00:00<?, ?file/s]

Reading files: 100%|██████████| 732/732 [00:54<00:00, 13.35file/s]


In [11]:
# รวม energy & temperature

import os
import pandas as pd
from tqdm import tqdm
from datetime import datetime

#read files
df1 = pd.read_csv(r"E:\Sort Data\cham4_mix\cham4_energy&temp&humidity_4.csv")  #<<<<<<<<<<<<<<
df2 = pd.read_csv(r"E:\Sort Data\cham4_intensity\cham4_intensity_4.csv")  #<<<<<<<<<<<<<<

# Convert 'time' columns to datetime objects for both DataFrames
df1['time'] = pd.to_datetime(df1['time'])
df2['time'] = pd.to_datetime(df2['time'])

# Sort both DataFrames by 'time'
df1 = df1.sort_values(by='time')
df2 = df2.sort_values(by='time')

# Merge the two DataFrames based on 'time' and 'id_floor' using merge_asof
result = pd.merge_asof(df1, df2, on='time', by='id_floor', direction='forward', tolerance=pd.Timedelta('5 minutes'))

#sort
result = result.sort_values(by=['id_floor','time'])

print(result)

result.to_csv(r"E:\Sort Data\cham4_4variable_4.csv", index=False)  #<<<<<<<<<<<<<<

       id_floor                      time  energy  temperature  humidity  \
0           fl1 2022-09-10 09:00:00+08:00   2.525        28.90     85.33   
12          fl1 2022-09-10 09:01:00+08:00   2.592        28.90     85.33   
15          fl1 2022-09-10 09:02:00+08:00   2.553        28.90     85.33   
22          fl1 2022-09-10 09:03:00+08:00   4.292        28.90     85.33   
30          fl1 2022-09-10 09:04:00+08:00  12.456        28.90     85.40   
...         ...                       ...     ...          ...       ...   
282284      fl7 2022-12-31 23:55:00+08:00  60.444        29.41     53.69   
282292      fl7 2022-12-31 23:56:00+08:00  60.462        29.89     50.75   
282302      fl7 2022-12-31 23:57:00+08:00  60.355        29.89     50.75   
282308      fl7 2022-12-31 23:58:00+08:00  55.183        29.51     49.74   
282316      fl7 2022-12-31 23:59:00+08:00  53.679        29.28     54.44   

        intensity  
0             0.0  
12            0.0  
15            0.0  
22     

In [4]:
import os
import pandas as pd
from tqdm import tqdm
from datetime import datetime

a=r"E:\Sort Data\cham4_energy&temp&humidity_1.csv"

df = pd.read_csv(a)

#df = df.sort_values(by=['id_floor','time'])

# df.rename(columns={'value': 'energy'}, inplace=True)
# df.rename(columns={'id_group': 'id_floor'}, inplace=True)

# Count the number of empty energy and temperature values
num_empty_energy = df['energy'].isnull().sum()
num_empty_temperature = df['temperature'].isnull().sum()
num_empty_humidity = df['humidity'].isnull().sum()

print(f"Number of empty energy values: {num_empty_energy}")
print(f"Number of empty temperature values: {num_empty_temperature}")
print(f"Number of empty humidity values: {num_empty_humidity}")

#df.to_csv(a, index=False)

Number of empty energy values: 0
Number of empty temperature values: 507
Number of empty humidity values: 507


In [16]:
import os
import pandas as pd
from tqdm import tqdm
from datetime import datetime

def read_files_in_fraction(directory_path, fraction_start, fraction_end):
    # Get a list of file names in the directory
    file_names = os.listdir(directory_path)

    # Calculate the number of files to read based on the fraction
    num_files_to_read_start = int(len(file_names) * fraction_start)
    num_files_to_read_end = int(len(file_names) * fraction_end)

    # Initialize an empty list to store individual DataFrames
    dataframes = []

    # Use tqdm to create a progress bar for reading files
    for file_name in tqdm(file_names[num_files_to_read_start:num_files_to_read_end], desc="Reading files", unit="file"):
        if file_name.endswith('.csv'):  # Adjust the file extension based on your files (e.g., .csv, .xlsx, etc.)
            file_path = os.path.join(directory_path, file_name)
            df2 = pd.read_csv(file_path)  # Replace 'read_csv' with 'read_excel' for Excel files
            dataframes.append(df2)

    # Concatenate all the individual DataFrames into a single DataFrame
    df3 = pd.concat(dataframes, ignore_index=True)

    return df3

# Replace 'directory_path' with the path to the directory containing your files
directory_path = 'E:\\CUSC Datalake\\cham4'  #<<<<<<<<<<<<<

# Read 1/4 of the files
df = read_files_in_fraction(directory_path, 0, 0.25)  #<<<<<<<<<<<<<

df = df[df['id'].str.contains('intensity')]

df['id_floor'] = df['id'].str.extract(r'(fl(?:m|\d{1,2}[a-zA-Z]?))')

df = df.sort_values(by=['id_floor','time'])

df.to_csv(r"E:\Sort Data\cham4_intensity_test.csv", index=False) 

Reading files: 100%|██████████| 732/732 [00:59<00:00, 12.21file/s]


In [7]:
#หาไอดีไม่ซ้ำ
import os
import pandas as pd
from tqdm import tqdm

df = pd.read_csv(r"E:\Sort Data\cham4_intensity\cham4_intensity_1.csv")

#unique_id_column = df.drop_duplicates(subset='id')['id']

df = df[df['intensity'] <= 10000]

df.to_csv(r"E:\Sort Data\cham4_intensity\cham4_intensity_new1.csv", index=False)